In [ ]:
# plotting contourf-plots for all depths
# Latitiude / Longitude and Hovmöller
# Temperature (RBR or CTD) and salinity (CTD)
# choose mission and Temperature sensor

# Pia Goecke, pia.goecke@uni-oldenburg.de
# 09.12.2024

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
import os
import cmocean
from matplotlib import cm

%matplotlib qt

#read data and rename cols
os.chdir('C:/Users/piago/Documents/Uni/5_Semester/Hiwi')
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE614_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file, skiprows=10)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df.rename(columns={'CTD_Temperature_SML' : 'CTD_Temperature_1',
                   'CTD_Temperature_30cm' : 'CTD_Temperature_30',
                   'CTD_Temperature_40cm' : 'CTD_Temperature_40',
                   'CTD_Temperature_50cm_[°C]' : 'CTD_Temperature_50',
                   'CTD_Temperature_60cm_[°C]': 'CTD_Temperature_60',
                   'CTD_Temperature_85cm' : 'CTD_Temperature_85',
                   'CTD_Temperature_100cm' : 'CTD_Temperature_100',
                   'RBR_Temperature_30cm' :  'RBR_Temperature_30',
                   'RBR_Temperature_40cm' :  'RBR_Temperature_40',
                   'RBR_Temperature_50cm' :  'RBR_Temperature_50',
                   'RBR_Temperature_60cm' :  'RBR_Temperature_60',
                   'RBR_Temperature_85cm' :  'RBR_Temperature_85',
                   'RBR_Temperature_100cm' :  'RBR_Temperature_100'
                   }, inplace= True)

df['Date'] = df.index.date
df_backup = df
# extract dfs
df_2 = df.loc['2023-03-02']
df_2 = df_2[(df_2.index < '2023-03-02 09:48') | (df_2.index >= '2023-03-02 09:49')]

df_3 = df.loc['2023-03-03']

df_8 = df.loc['2023-03-08']
df_8 = df_8[(df_8.index < '2023-03-08 09:19:10') | (df_8.index >= '2023-03-08 09:21:40')]


df_9 = df.loc['2023-03-09']
df_9.drop(df_9[df_9['CTD_Temperature_1']>6.2].index, inplace = True)


df_10 = df.loc['2023-03-10']

df_11 = df.loc['2023-03-11']

df_17 = df.loc['2023-03-17']
df_17 = df_17[df_17.index > '2023-03-17 08:03']

df_18 = df.loc['2023-03-18']
df_18 = df_18[df_18.index > '2023-03-18 07:50']

df_19 = df.loc['2023-03-19']
df_19 = df_19[(df_19.index < '2023-03-08 10:15:30') | (df_19.index >= '2023-03-19 10:16:20')]

df_20 = df.loc['2023-03-20']


all_df = [df_2, df_3, df_8, df_9, df_10, df_11, df_17, df_18, df_19, df_20]
#Detrend CTD, RBR and Campbell Air temp where reasonable

for df in [df_2, df_9, df_17]:
    # time in total seconds
    df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

    # fit polynomial with numpy.polyfit:  quadratic
    coefficients = np.polyfit(df['time_numeric'], df['CTD_Temperature_1'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_SML'] = polynomial(df['time_numeric']) #values
    df[f'detrend_2_Temp_SML'] = df['CTD_Temperature_1'] - df[f'fitted_2_Temp_SML']
    df[f'detrend_2_Temp_SML'] = df[f'detrend_2_Temp_SML'] + df[f'fitted_2_Temp_SML'].median()

    # same for all depths
    depths = [30, 40, 50, 60, 85, 100]

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"Column CTD_Temperature_{d}cm_[°C] contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()


    # ... and same for airtemp
    coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
    df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
    df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

    # ... and for RBRs
    depths = [30, 40, 50, 60, 85, 100]
    for d in depths:
        if df[f'RBR_Temperature_{d}'].dropna().empty:
            print(f"Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
        else: 
            coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

    print(f'mission {str(df['Date'][0])} detrended.')
df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]

Column CTD_Temperature_40cm_[°C] contains only NaN values. Creating nan-column for depth 40.
mission 2023-03-02 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-09 detrended.
Column CTD_Temperature_50cm_[°C] contains only NaN values. Creating nan-column for depth 50.
mission 2023-03-17 detrended.


In [2]:
df = df_9_1 # which mission to work with. df = df_3[df_3.index<'2023-03-03 13:00:00'] 

df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

In [3]:
# create df_temp for plotting. choose temperature sensor

sensor = 'CTD' # 'RBR', or 'CTD'

if sensor == 'RBR':
    depths = [30, 40, 50, 60, 85, 100]
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended RBR data is used')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'rbr_detrend_2_Temp_30', 'rbr_detrend_2_Temp_40','rbr_detrend_2_Temp_50',  'rbr_detrend_2_Temp_60', 
                 'rbr_detrend_2_Temp_85', 'rbr_detrend_2_Temp_100', 'Air_detrend_2_Temp']]

        df_temp.rename(columns={
                        'rbr_detrend_2_Temp_30' : 'RBR_Temperature_30',
                        'rbr_detrend_2_Temp_40' : 'RBR_Temperature_40',
                        'rbr_detrend_2_Temp_50' : 'RBR_Temperature_50',
                        'rbr_detrend_2_Temp_60' : 'RBR_Temperature_60',
                        'rbr_detrend_2_Temp_85' : 'RBR_Temperature_85',
                        'rbr_detrend_2_Temp_100' : 'RBR_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original RBR data used')
        df_temp = df.reset_index()

    vmin_temp = df_temp[['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50', 'RBR_Temperature_60', 
           'RBR_Temperature_85', 'RBR_Temperature_100']].min(axis=1).min()
    vmax_temp = df_temp[['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50', 'RBR_Temperature_60', 
            'RBR_Temperature_85', 'RBR_Temperature_100']].max(axis=1).max()

elif sensor == 'CTD':
    depths = [1, 30, 40, 50, 60, 85, 100]
    if 'Xdetrend_2_Temp_SML' in df.columns:
        print('detrended CTD data is used')
        df_temp = df.reset_index()
        df_temp = df_temp[['Latitude', 'Longitude', 'detrend_2_Temp_SML', 'detrend_2_Temp_30', 'detrend_2_Temp_40',
                    'detrend_2_Temp_50',  'detrend_2_Temp_60', 'detrend_2_Temp_85', 'detrend_2_Temp_100', 'Air_detrend_2_Temp' ]]

        df_temp.rename(columns={'detrend_2_Temp_SML' : 'CTD_Temperature_1',
                        'detrend_2_Temp_30' : 'CTD_Temperature_30',
                        'detrend_2_Temp_40' : 'CTD_Temperature_40',
                        'detrend_2_Temp_50' : 'CTD_Temperature_50',
                        'detrend_2_Temp_60': 'CTD_Temperature_60',
                        'detrend_2_Temp_85' : 'CTD_Temperature_85',
                        'detrend_2_Temp_100' : 'CTD_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original CTD data used')
        df_temp = df.reset_index()

    vmin_temp = df_temp[['CTD_Temperature_1', 'CTD_Temperature_30', 'CTD_Temperature_40', 'CTD_Temperature_50', 'CTD_Temperature_60', 
           'CTD_Temperature_85', 'CTD_Temperature_100']].min(axis=1).min()
    vmax_temp = df_temp[['CTD_Temperature_1', 'CTD_Temperature_30', 'CTD_Temperature_40', 'CTD_Temperature_50', 'CTD_Temperature_60', 
            'CTD_Temperature_85', 'CTD_Temperature_100']].max(axis=1).max()


original CTD data used


In [4]:
# create df_sal for plotting

df_sal = df.reset_index()
df_sal.rename(columns={'CTD_Salinity_SML' : 'CTD_Salinity_1',
                'CTD_Salinity_30cm' : 'CTD_Salinity_30',
                'CTD_Salinity_40cm' : 'CTD_Salinity_40',
                'CTD_Salinity_practical_50cm_[ ]' : 'CTD_Salinity_50',
                'CTD_Salinity_practical_60cm_[ ]' : 'CTD_Salinity_60',
                'CTD_Salinity_85cm' : 'CTD_Salinity_85',
                'CTD_Salinity_100cm' : 'CTD_Salinity_100'}, inplace= True)

vmin_sal = df_sal[[
#     'CTD_Salinity_1', 
    'CTD_Salinity_30', 'CTD_Salinity_40', 'CTD_Salinity_50', 'CTD_Salinity_60', 
        'CTD_Salinity_85', 
        'CTD_Salinity_100']].min(axis=1).min()
vmax_sal = df_sal[[
#     'CTD_Salinity_1', 
    'CTD_Salinity_30', 'CTD_Salinity_40', 'CTD_Salinity_50', 'CTD_Salinity_60', 
        'CTD_Salinity_85', 
        'CTD_Salinity_100']].max(axis=1).max()



In [5]:
# plot temperature-contourf for all depths

for depth in depths:
    df_loop = df_temp.dropna(subset= [f'{sensor}_Temperature_{depth}'])

    try:
        xi = np.linspace(min(df_loop.Longitude), max(df_loop.Longitude))
        yi = np.linspace(min(df_loop.Latitude),max(df_loop.Latitude))

        zi = griddata((df_loop.Longitude,df_loop.Latitude), df_temp[f'{sensor}_Temperature_{depth}'], (xi[None,:], yi[:,None]), method='linear')
        plt.figure(figsize=(8,5))
        plt.grid()
        C = plt.contourf(xi,yi,zi, 15,cmap='coolwarm'      #cmocean.cm.thermal 
                         , vmin = vmin_temp, vmax = vmax_temp
                         )
#
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        
        plt.ylabel("Latitude", fontsize=20)
        plt.xlabel("Longitude", fontsize=20)
        # plt.title(f'Temperature_Spatial {sensor} {depth}',fontsize=25, fontweight='bold')
        plt.title(f'Temperature SML',fontsize=25, fontweight='bold')

        # plt.colorbar(C, label='Temperature [°C]')
        cbar = plt.colorbar(C, cmap='coolwarm', pad=0.03)
        cbar.set_label('Temperature [°C]', fontsize=20)
        plt.show()
        plt.show()
    except ValueError:
        print(f'No values for depth {depth}. Continue with next depth.')
        continue



No values for depth 50. Continue with next depth.


In [6]:
# wind speed contourf
df_plot = df_17

xi = np.linspace(min(df_plot.Longitude), max(df_plot.Longitude))
yi = np.linspace(min(df_plot.Latitude),max(df_plot.Latitude))

zi = griddata((df_plot.Longitude,df_plot.Latitude),df_plot['Wind_Speed_U10'], (xi[None,:], yi[:,None]), method='linear')
plt.figure(figsize=(8,5))
plt.grid()
C = plt.contourf(xi,yi,zi, 15,cmap=cm.viridis
                    # , vmin = vmin_temp, vmax = vmax_temp
                    )

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.ylabel("Latitude", fontsize=18, fontweight='bold')
plt.xlabel("Longitude", fontsize=18, fontweight='bold')
plt.title(f'Wind Speed 10 m [m/s]',fontsize=22, fontweight='bold')
plt.colorbar(C, label='Wind Speed 10 m [m/s]')
plt.show()

In [7]:
# temperature difference 100cm - SML
df_plot = df_temp
df_plot['100-SML_Temp'] = df_plot[f'CTD_Temperature_100'] - df_plot[f'CTD_Temperature_1']

xi = np.linspace(min(df_plot.Longitude), max(df_plot.Longitude))
yi = np.linspace(min(df_plot.Latitude),max(df_plot.Latitude))

zi = griddata((df_plot.Longitude,df_plot.Latitude),df_plot['100-SML_Temp'], (xi[None,:], yi[:,None]), method='linear')
plt.figure(figsize=(8,5))
plt.grid()
C = plt.contourf(xi,yi,zi, 15,cmap=cmocean.cm.thermal
                    # , vmin = vmin_temp, vmax = vmax_temp
                    )

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.ylabel("Latitude", fontsize=18, fontweight='bold')
plt.xlabel("Longitude", fontsize=18, fontweight='bold')
plt.title(f'Temperature_Spatial {sensor} 100cm - SML',fontsize=22, fontweight='bold')
plt.colorbar(C, label='Temperature difference [°C]')
plt.show()

In [8]:
# plot salinity-contourf for all depths (practical [PSU])

depths = [1, 30, 40, 50, 60, 85, 100]
for depth in depths:
    df_loop = df_sal.dropna(subset= [f'CTD_Salinity_{depth}'])

    try:
        xi = np.linspace(min(df_loop.Longitude), max(df_loop.Longitude))
        yi = np.linspace(min(df_loop.Latitude),max(df_loop.Latitude))

        zi = griddata((df_loop.Longitude,df_sal.Latitude), df_loop[f'CTD_Salinity_{depth}'], (xi[None,:], yi[:,None]), method='linear')
        plt.figure(figsize=(8,5))
        plt.grid()
        C = plt.contourf(xi,yi,zi, 15,cmap=cmocean.cm.haline
                         , vmin = vmin_sal, vmax = vmax_sal
                         )

        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        
        plt.ylabel("Latitude", fontsize=18, fontweight='bold')
        plt.xlabel("Longitude", fontsize=18, fontweight='bold')
        plt.title(f'Salinity_Spatial {depth}',fontsize=22, fontweight='bold')
        plt.colorbar(C, label='Salinity [PSU]')
        plt.show()
    except ValueError:
        print(f'No values for depth {depth}. Continue with next depth.')
        continue

No values for depth 50. Continue with next depth.


In [ ]:
###########################

In [9]:
df = df_3
df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

In [10]:
# create df_temp for plotting Hovmöller (time). choose temperature sensor

sensor = 'CTD' # 'RBR', or 'CTD'

if sensor == 'RBR':
    depths = [30, 40, 50, 60, 85, 100]
    if 'detrend_2_Temp_SML' in df.columns:
        print('detrended RBR data is used')
        df_temp_time = df.reset_index()
        df_temp_time = df_temp_time[['Latitude', 'Longitude', 'time_numeric', 'rbr_detrend_2_Temp_30', 'rbr_detrend_2_Temp_40','rbr_detrend_2_Temp_50',  'rbr_detrend_2_Temp_60', 
                 'rbr_detrend_2_Temp_85', 'rbr_detrend_2_Temp_100', 'Air_detrend_2_Temp']]

        df_temp_time.rename(columns={
                        'rbr_detrend_2_Temp_30' : 'RBR_Temperature_30',
                        'rbr_detrend_2_Temp_40' : 'RBR_Temperature_40',
                        'rbr_detrend_2_Temp_50' : 'RBR_Temperature_50',
                        'rbr_detrend_2_Temp_60' : 'RBR_Temperature_60',
                        'rbr_detrend_2_Temp_85' : 'RBR_Temperature_85',
                        'rbr_detrend_2_Temp_100' : 'RBR_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original RBR data used')
        df_temp_time = df.reset_index()

    vmin = df_temp_time[['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50', 'RBR_Temperature_60', 
           'RBR_Temperature_85', 'RBR_Temperature_100']].min(axis=1).min()
    vmax = df_temp_time[['RBR_Temperature_30', 'RBR_Temperature_40', 'RBR_Temperature_50', 'RBR_Temperature_60', 
            'RBR_Temperature_85', 'RBR_Temperature_100']].max(axis=1).max()

elif sensor == 'CTD':
    depths = [1, 30, 40, 50, 60, 85, 100]
    if 'Odetrend_2_Temp_SML' in df.columns:
        print('detrended CTD data is used')
        df_temp_time = df.reset_index()
        df_temp_time = df_temp_time[['Latitude', 'Longitude', 'time_numeric', 'detrend_2_Temp_SML', 'detrend_2_Temp_30', 'detrend_2_Temp_40',
                    'detrend_2_Temp_50',  'detrend_2_Temp_60', 'detrend_2_Temp_85', 'detrend_2_Temp_100', 'Air_detrend_2_Temp' ]]

        df_temp_time.rename(columns={'detrend_2_Temp_SML' : 'CTD_Temperature_1',
                        'detrend_2_Temp_30' : 'CTD_Temperature_30',
                        'detrend_2_Temp_40' : 'CTD_Temperature_40',
                        'detrend_2_Temp_50' : 'CTD_Temperature_50',
                        'detrend_2_Temp_60': 'CTD_Temperature_60',
                        'detrend_2_Temp_85' : 'CTD_Temperature_85',
                        'detrend_2_Temp_100' : 'CTD_Temperature_100',
                        'Air_detrend_2_Temp' : 'Air_Temperature_999'}, inplace= True)
    else:
        print('original CTD data used')
        df_temp_time = df.reset_index()

    vmin_temp = df_temp_time[['CTD_Temperature_1', 'CTD_Temperature_30', 'CTD_Temperature_40', 'CTD_Temperature_50', 'CTD_Temperature_60', 
           'CTD_Temperature_85', 'CTD_Temperature_100']].min(axis=1).min()
    vmax_temp = df_temp_time[['CTD_Temperature_1', 'CTD_Temperature_30', 'CTD_Temperature_40', 'CTD_Temperature_50', 'CTD_Temperature_60', 
            'CTD_Temperature_85', 'CTD_Temperature_100']].max(axis=1).max()


original CTD data used


In [11]:
# plot temperature-contourf for all depths, time-latitude

for depth in depths:
    df_loop = df_temp_time.dropna(subset= [f'{sensor}_Temperature_{depth}'])

    try:
        xi = np.linspace(min(df_loop.time_numeric), max(df_loop.time_numeric))
        yi = np.linspace(min(df_loop.Latitude),max(df_loop.Latitude))

        zi = griddata((df_loop.time_numeric,df_loop.Latitude), df_loop[f'{sensor}_Temperature_{depth}'], (xi[None,:], yi[:,None]), method='linear')
        plt.figure(figsize=(8,5))
        plt.grid()
        C = plt.contourf(xi,yi,zi, 15,cmap=cmocean.cm.thermal
                         , vmin = vmin_temp, vmax = vmax_temp
                         )

        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        
        plt.ylabel("Latitude [°]", fontsize=18, fontweight='bold')
        plt.xlabel("Time [s]", fontsize=18, fontweight='bold')
        plt.title(f'Temperature_Spatial-time {sensor} {depth}',fontsize=22, fontweight='bold')
        plt.colorbar(C, label='Temperature [°C]')
        plt.show()
    except ValueError:
        print(f'No values for depth {depth}. Continue with next depth.')
        continue



In [29]:
# plot temperature-contourf for all depths, longitude-temp

for depth in depths:
    df_loop = df_temp_time.dropna(subset= [f'{sensor}_Temperature_{depth}'])

    try:
        xi = np.linspace(min(df_loop.Longitude), max(df_loop.Longitude))
        yi = np.linspace(min(df_loop.time_numeric),max(df_loop.time_numeric))

        zi = griddata((df_loop.Longitude,df_loop.time_numeric), df_loop[f'{sensor}_Temperature_{depth}'], (xi[None,:], yi[:,None]), method='linear')
        plt.figure(figsize=(8,5))
        plt.grid()
        C = plt.contourf(xi,yi,zi, 15,cmap='coolwarm'   #cmocean.cm.thermal #
                         , vmin = vmin_temp, vmax = vmax_temp
                         )

        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        
        plt.xlabel("Longitude", fontsize=18)
        plt.ylabel("Time [s]", fontsize=18)
        #plt.title(f'Temperature_Spatial-time {sensor} {depth}',fontsize=22, fontweight='bold')
        plt.title(f'Temperature SML',fontsize=22, fontweight='bold')
        # plt.colorbar(C, label='Temperature [°C]')

        cbar = plt.colorbar(C, cmap='coolwarm', pad=0.03)
        cbar.set_label('Temperature [°C]', fontsize=20)
        plt.gca().invert_yaxis()
        plt.show()
    except ValueError:
        print(f'No values for depth {depth}. Continue with next depth.')
        continue



No values for depth 50. Continue with next depth.
